<font color='green'>***Installation and Libraries Import***</font>
---
--- 

In [75]:

# %pip install flwr
# %pip install ray 
# %pip install --upgrade pip
# %pip install torch torchvision matplotlib
# %pip install async-timeout
# %pip install async-numpy
# %pip install pandas
# %pip install datasets
# %pip install scikit-learn


In [76]:
import os
import flwr as fl
import torch, ray, pandas, sklearn
import pickle
from pprint import pprint
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import ConcatDataset, DataLoader


# print("All modules loaded successfully!")
# print("FLWR version:", fl.__version__)
# print("Ray version:", ray.__version__)
# print("Torch version:", torch.__version__)

# libraries_to_uninstall = [
#     "tb-nightly==2.18.0a20240701",
#     "tensorboard==2.16.2",
#     "tensorboard-data-server==0.7.2",
#     "tensorboard-plugin-wit==1.8.1",
#     "tensorflow==2.16.2",
#     "tensorflow-io-gcs-filesystem==0.37.0",
#     "termcolor==2.4.0",
#     "terminado==0.18.1",
#     "tf-estimator-nightly==2.8.0.dev2021122109",
#     "tf_keras-nightly==2.18.0.dev2024070109",
#     "tf-nightly==2.18.0.dev20240626"
# ]
# for library in libraries_to_uninstall:
#     os.system(f"pip uninstall -y {library}")
# print("All modules loaded successfully!")
# print("FLWR version:", fl.__version__)
# print("Ray version:", ray.__version__)
# print("Torch version:", torch.__version__)

In [77]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader, random_split, Subset
import torch.optim as optim
from torch.utils.data import Dataset, WeightedRandomSampler
import flwr as fl
from flwr.common import Metrics
# from flwr_datasets import FederatedDataset
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
import random
import math
import pickle
import csv
import copy
print(fl.__version__)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

1.5.0
cuda:0


<font color='Brown'>***Constants***</font>
---
--- 

In [78]:
# DEVICE
NUM_CLIENTS = 48 #48
ROUNDS = 40 #40
BATCH_SIZE = 64 #C
LEARNING_RATE = 0.0025 #C
EPOCHS = 3 #C
DATA_GROUPS = 500
BATCH_ROUND = 6
SIZE_ROUND = int(BATCH_ROUND * BATCH_SIZE * NUM_CLIENTS)
NUM_ATCKS = 11
FL = True # FL or CENT
PATH = f'FL:{FL}DNN{NUM_CLIENTS}-clients-{NUM_ATCKS}atk-{ROUNDS}-rounds-{EPOCHS}-epochs-{LEARNING_RATE}-lr-{DATA_GROUPS}-groups'
G = 0

<font color='Light Blue'>***Dataset Preparations***</font>
---
--- 

In [79]:
TrafficData = {}
TrafficData['Dataset']={}
sets_names = ['30','100','70','50','120']
for  DATA_NUM in sets_names:
    TrafficData['Dataset'][DATA_NUM]=pd.read_csv(f'data/2_Dataset_{NUM_ATCKS}_Attack_{DATA_NUM}_normal.csv', low_memory=False, quoting=csv.QUOTE_NONE, on_bad_lines='skip')
    print(DATA_NUM, TrafficData['Dataset'][DATA_NUM].shape)
for DATA_NUM in TrafficData['Dataset']:
    TrafficData['Dataset'][DATA_NUM]=TrafficData['Dataset'][DATA_NUM].sample(frac=1, random_state=42).reset_index(drop=True)

30 (184320, 99)
100 (184320, 99)
70 (184320, 99)
50 (184320, 99)
120 (120000, 99)


In [80]:
TrafficData['Split'] = {}
sets_training =  ['30','100','70','50']
for DATA_NUM in sets_training:
    TrafficData['Split'][DATA_NUM] = np.array_split(TrafficData['Dataset'][DATA_NUM],DATA_GROUPS)

TrafficData['Combined'] = pd.concat([TrafficData['Split']['30'][0], TrafficData['Split']['100'][0], TrafficData['Split']['70'][0], TrafficData['Split']['50'][0]]).reset_index(drop=True)
for GROUP in range(1, DATA_GROUPS):
    TrafficData['Combined'] = pd.concat([TrafficData['Combined'], TrafficData['Split']['30'][GROUP], TrafficData['Split']['100'][GROUP], TrafficData['Split']['70'][GROUP], TrafficData['Split']['50'][GROUP]]).reset_index(drop=True)
print(TrafficData['Combined'].shape)            

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


(737280, 99)


In [81]:
TrafficData['Train'] = {}
TrafficData['Train']['X'] = TrafficData['Combined'].iloc[:, 0:-1]
TrafficData['Train']['y'] = TrafficData['Combined'].iloc[:, -1]
print(TrafficData['Train']['X'].shape)
print(TrafficData['Train']['y'].shape)

TrafficData['Test'] = {}
TrafficData['Test']['X']=TrafficData['Dataset']['120'].iloc[:, 0:-1]
TrafficData['Test']['y']=TrafficData['Dataset']['120'].iloc[:, -1]
print(TrafficData['Test']['X'].shape)
print(TrafficData['Test']['y'].shape)

(737280, 98)
(737280,)
(120000, 98)
(120000,)


In [82]:
scaler = MinMaxScaler()
model = scaler.fit(TrafficData['Train']['X'])
TrafficData['Train']['X'] = model.transform(TrafficData['Train']['X'])
TrafficData['Test']['X'] = model.transform(TrafficData['Test']['X'])

TrafficData['Train']['X'], TrafficData['Train']['y']= np.array(TrafficData['Train']['X']), np.array(TrafficData['Train']['y'])
print(type(TrafficData['Train']['X']),type(TrafficData['Train']['y']))
print(TrafficData['Train']['X'].shape,TrafficData['Train']['y'].shape)
TrafficData['Test']['X'], TrafficData['Test']['y']= np.array(TrafficData['Test']['X']), np.array(TrafficData['Test']['y'])
print(type(TrafficData['Test']['X']),type(TrafficData['Test']['y']))
print(TrafficData['Test']['X'].shape,TrafficData['Test']['y'].shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(737280, 98) (737280,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(120000, 98) (120000,)


In [83]:
TrafficData['ROUNDS']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['ROUNDS'][ROUND]={}

SIZE_Demo = SIZE_ROUND
for ROUND in range(1,ROUNDS+1):
    if ROUND == 1:
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][:SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][:SIZE_Demo]
    else:
        print((SIZE_Demo - SIZE_ROUND),SIZE_Demo)
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
    SIZE_Demo = SIZE_Demo + SIZE_ROUND
for ROUND in TrafficData['ROUNDS']:
    print("ROUND: ", ROUND, TrafficData['ROUNDS'][ROUND]['X'].shape, TrafficData['ROUNDS'][ROUND]['y'].shape)
print(SIZE_Demo, SIZE_ROUND)

18432 36864
36864 55296
55296 73728
73728 92160
92160 110592
110592 129024
129024 147456
147456 165888
165888 184320
184320 202752
202752 221184
221184 239616
239616 258048
258048 276480
276480 294912
294912 313344
313344 331776
331776 350208
350208 368640
368640 387072
387072 405504
405504 423936
423936 442368
442368 460800
460800 479232
479232 497664
497664 516096
516096 534528
534528 552960
552960 571392
571392 589824
589824 608256
608256 626688
626688 645120
645120 663552
663552 681984
681984 700416
700416 718848
718848 737280
ROUND:  1 (18432, 98) (18432,)
ROUND:  2 (18432, 98) (18432,)
ROUND:  3 (18432, 98) (18432,)
ROUND:  4 (18432, 98) (18432,)
ROUND:  5 (18432, 98) (18432,)
ROUND:  6 (18432, 98) (18432,)
ROUND:  7 (18432, 98) (18432,)
ROUND:  8 (18432, 98) (18432,)
ROUND:  9 (18432, 98) (18432,)
ROUND:  10 (18432, 98) (18432,)
ROUND:  11 (18432, 98) (18432,)
ROUND:  12 (18432, 98) (18432,)
ROUND:  13 (18432, 98) (18432,)
ROUND:  14 (18432, 98) (18432,)
ROUND:  15 (18432, 98) (

In [84]:
class ClassifierDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

In [85]:
TrafficData['trainsets']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['trainsets'][ROUND]= ClassifierDataset(torch.from_numpy(TrafficData['ROUNDS'][ROUND]['X']).float(), torch.from_numpy(TrafficData['ROUNDS'][ROUND]['y']).long())
TrafficData['testset'] = ClassifierDataset(torch.from_numpy(TrafficData['Test']['X']).float(), torch.from_numpy(TrafficData['Test']['y']).long())

In [86]:
def load_train(numberofclients, ROUND):    
    # Use the actual dataset length to compute balanced portions per client
    dataset_len = len(TrafficData['trainsets'][ROUND])
    if dataset_len == 0:
        # Return an empty list of loaders to avoid indexing errors downstream
        return [DataLoader(Subset(TrafficData['trainsets'][ROUND], []), batch_size=BATCH_SIZE, shuffle=False) for _ in range(numberofclients)]
    
    # Distribute samples as evenly as possible across clients (handle remainders)
    num_portions = int(numberofclients)
    base_portion = dataset_len // num_portions
    remainder = dataset_len % num_portions
    portion_indices = []
    start_idx = 0
    for i in range(num_portions):
        # First `remainder` clients receive one extra sample
        sz = base_portion + (1 if i < remainder else 0)
        end_idx = start_idx + sz
        if sz > 0:
            portion_indices.append(list(range(start_idx, end_idx)))
        else:
            portion_indices.append([])
        start_idx = end_idx
    
    # Build Subset and DataLoader for each client (safe: indices are within [0, dataset_len))
    portion_datasets = [Subset(TrafficData['trainsets'][ROUND], indices) for indices in portion_indices]
    portion_loaders = [DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False) for dataset in portion_datasets]            
    return portion_loaders
def load_test(numberofclients):    
    testloader = DataLoader(TrafficData['testset'], batch_size=BATCH_SIZE, shuffle=False)
    return testloader

In [87]:
Dataloaders = {}
for ROUND in range(1, ROUNDS+1):
    Dataloaders[ROUND] = load_train(NUM_CLIENTS, ROUND)
Dataloaders['Test'] = load_test(NUM_CLIENTS)

In [88]:
# Diagnostic: print per-client dataset sizes for the first few rounds to verify splits
for ROUND in range(1, min(6, ROUNDS+1)):
    loaders = Dataloaders.get(ROUND, None)
    if loaders is None:
        print(f"Dataloaders for Round {ROUND} not found (run the rebuild cell).")
        continue
    sizes = [len(loader.dataset) for loader in loaders]
    print(f"Round {ROUND}: per-client sizes (first 12): {sizes[:12]}")
    print(f"  Total samples this round: {sum(sizes)}\n")

Round 1: per-client sizes (first 12): [384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384]
  Total samples this round: 18432

Round 2: per-client sizes (first 12): [384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384]
  Total samples this round: 18432

Round 3: per-client sizes (first 12): [384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384]
  Total samples this round: 18432

Round 4: per-client sizes (first 12): [384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384]
  Total samples this round: 18432

Round 5: per-client sizes (first 12): [384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384]
  Total samples this round: 18432



In [89]:
for i, batch in enumerate(Dataloaders['Test']):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64
Batch 7 size: 64
Batch 8 size: 64
Batch 9 size: 64
Batch 10 size: 64
Batch 11 size: 64
Batch 12 size: 64
Batch 13 size: 64
Batch 14 size: 64
Batch 15 size: 64
Batch 16 size: 64
Batch 17 size: 64
Batch 18 size: 64
Batch 19 size: 64
Batch 20 size: 64
Batch 21 size: 64
Batch 22 size: 64
Batch 23 size: 64
Batch 24 size: 64
Batch 25 size: 64
Batch 26 size: 64
Batch 27 size: 64
Batch 28 size: 64
Batch 29 size: 64
Batch 30 size: 64
Batch 31 size: 64
Batch 32 size: 64
Batch 33 size: 64
Batch 34 size: 64
Batch 35 size: 64
Batch 36 size: 64
Batch 37 size: 64
Batch 38 size: 64
Batch 39 size: 64
Batch 40 size: 64
Batch 41 size: 64
Batch 42 size: 64
Batch 43 size: 64
Batch 44 size: 64
Batch 45 size: 64
Batch 46 size: 64
Batch 47 size: 64
Batch 48 size: 64
Batch 49 size: 64
Batch 50 size: 64
Batch 51 size: 64
Batch 52 size: 64
Batch 53 size: 64
Batch 54 size: 64
Batch 55 size: 64
Batch 56 size: 64
B

In [90]:
for i, batch in enumerate(Dataloaders[5][0]):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64


In [91]:
from collections import Counter
for CLUSTER in range (1, 9):
    DEVICE_PERCENTAGE = []
    for DEVICE__ in range(0,NUM_CLIENTS):
        for i, batch in enumerate(Dataloaders[CLUSTER][DEVICE__]):
            _, labels = batch
            class_counts = Counter(labels.numpy())
            total_records = sum(class_counts.values())
            class_0_count = class_counts.get(0, 0)
            percentage_class_0 = (class_0_count / total_records) * 100
            DEVICE_PERCENTAGE.append(percentage_class_0)
            # print(f"Batch {i+1}: {dict(class_counts)}")
            # print(f"Percentage of class 0: {percentage_class_0:.2f}%\n")
    # print(DEVICE_PERCENTAGE)        
    chunk_size = 6
    averages = [sum(DEVICE_PERCENTAGE[i:i + chunk_size]) / chunk_size for i in range(0, len(DEVICE_PERCENTAGE), chunk_size)]
    # print("Averages of every device:")
    # print(averages)
    chunk_size_4 = 4
    averages = [sum(averages[i:i + chunk_size_4]) / chunk_size_4 for i in range(0, len(averages), chunk_size_4)]
    # print("Averages of every 4 devices:")
    print(averages)

[62.760416666666664, 60.872395833333336, 60.67708333333333, 62.23958333333333, 62.6953125, 60.872395833333336, 63.411458333333336, 64.38802083333333, 63.606770833333336, 63.997395833333336, 63.0859375, 63.34635416666667]
[63.73697916666667, 63.60677083333333, 64.0625, 63.28125, 60.611979166666664, 64.6484375, 62.23958333333333, 61.588541666666664, 63.02083333333333, 60.7421875, 62.5, 60.546875]
[60.546875, 60.9375, 59.9609375, 63.34635416666667, 61.653645833333336, 62.369791666666664, 61.848958333333336, 62.5, 63.997395833333336, 63.15104166666667, 64.453125, 63.216145833333336]
[63.020833333333336, 64.71354166666667, 61.5234375, 64.51822916666667, 60.546875, 63.99739583333333, 61.848958333333336, 63.411458333333336, 61.263020833333336, 62.109375, 63.34635416666667, 60.9375]
[62.890625, 61.197916666666664, 59.635416666666664, 61.5234375, 60.807291666666664, 61.39322916666667, 61.5234375, 63.28125, 63.541666666666664, 62.76041666666667, 63.671875, 62.76041666666667]
[63.671875, 63.02083

In [92]:
del TrafficData

<font color='Red'>***Neural Network***</font>
---
--- 

In [93]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()        
        self.layer_1 = nn.Linear(98, 64)
        self.layer_2 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, 15) 
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.layer_out(x)        
        return x

In [94]:
# Random  = Net()
# for param_tensor in Random.state_dict():
#     print(param_tensor, "\t", Random.state_dict()[param_tensor].size())
# torch.save(Random.state_dict(), "0_Input_Random_model_Net.pth")

In [95]:
def train(net, trainloader, epochs: int, verbose=True):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    net.train()
    prediction_matrix = []
    actual_matrix = []
    acc_matrix = []
    loss_matrix = []
    # Safeguard: if the trainloader has no samples, skip training
    try:
        dataset_len = len(trainloader.dataset)
    except Exception:
        dataset_len = 0
    if dataset_len == 0:
        if verbose:
            print("Warning: trainloader has 0 samples, skipping training.")
        return prediction_matrix, actual_matrix, acc_matrix, loss_matrix
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Accumulate loss as scalar sum over samples
            epoch_loss += loss.item() * labels.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
            predictions = torch.max(outputs.data, 1)[1]
            prediction_matrix.append(predictions.tolist())
            actual_matrix.append(labels.tolist())
        # Avoid division by zero if total==0 for safety
        if total == 0:
            epoch_loss_val = 0.0
            epoch_acc = 0.0
        else:
            epoch_loss_val = epoch_loss / total
            epoch_acc = correct / total
        loss_matrix.append(epoch_loss_val)
        acc_matrix.append(epoch_acc)
        if verbose:
            print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss_val:.6f} | Acc: {epoch_acc:.4f}")
    return prediction_matrix, actual_matrix, acc_matrix, loss_matrix
def test(net, testloader):
    criterion = nn.CrossEntropyLoss()
    correct, total, loss_sum = 0, 0, 0.0
    net.eval()
    prediction_matrix = []
    actual_matrix = []
    acc_matrix = []
    loss_matrix = []
    try:
        test_len = len(testloader.dataset)
    except Exception:
        test_len = 0
    if test_len == 0:
        print("Warning: testloader has 0 samples, skipping evaluation.")
        return 0.0, 0.0, prediction_matrix, actual_matrix, acc_matrix, loss_matrix
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss_sum += criterion(outputs, labels).item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    if total == 0:
        avg_loss = 0.0
        accuracy = 0.0
    else:
        avg_loss = loss_sum / total
        accuracy = correct / total
    loss_matrix.append(avg_loss)
    acc_matrix.append(accuracy)
    print(f"Evaluation: eval loss {avg_loss}, eval accuracy {accuracy}")
    return avg_loss, accuracy, prediction_matrix, actual_matrix, acc_matrix, loss_matrix

In [96]:
prediction_dict= {}
actual_dict= {}
accuracy_dict= {}
loss_dict= {}

<font color='Brown'>***Federated Learning Classes***</font>
---
--- 

In [97]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [98]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, FL_Update):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.FL_Update = FL_Update
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)
    def fit(self, parameters, config):
        local_epochs = config["local_epochs"]
        print(f"[Client {self.cid}, round {self.FL_Update}] fit, config: {config}")
        set_parameters(self.net, parameters)
        _1, _2, _3, _4 = train(self.net, self.trainloader, epochs=local_epochs)
        prediction_dict[f'C{self.cid}R{self.FL_Update}'] = _1
        actual_dict[f'C{self.cid}R{self.FL_Update}'] = _2
        accuracy_dict[f'C{self.cid}R{self.FL_Update}'] = _3
        loss_dict[f'C{self.cid}R{self.FL_Update}'] = _4
        # Save model updates (parameters)
        # update_filename = f'EdgeCooperation/Performance/Results/C{self.cid}R{self.FL_Update}_update.pkl'
        # with open(update_filename, 'wb') as update_outfile:
        #     pickle.dump(get_parameters(self.net), update_outfile)        

        return get_parameters(self.net), len(self.trainloader), {}
    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}")
        print(f"[Client {self.cid}] evaluate, config: {config}")
        return None

In [99]:
def retrieve_and_sort_client_updates(global_model, round_number, client_ids):
    client_updates = {}
    for cid in client_ids:
        update_filename = f'EdgeCooperation/Performance/Results/C{cid}R{round_number}_update.pkl'
        with open(update_filename, 'rb') as update_file:
            client_update = pickle.load(update_file)
            client_updates[cid] = client_update
    client_contributions = {cid: calculate_weight_magnitude(global_model, update) for cid, update in client_updates.items()}
    sorted_clients = sorted(client_contributions.items(), key=lambda x: x[1], reverse=True)
    least_contributing_clients = sorted_clients[-3:]
    return sorted_clients, least_contributing_clients

def calculate_weight_magnitude(global_model, client_update):
    """
    Calculate the L2 norm of the weight difference between the global model and client's updated model.
    
    Args:
    global_model (nn.Module): The global model before client update.
    client_update (list): List of numpy arrays representing client's updated model parameters.

    Returns:
    float: The L2 norm of the weight difference.
    """
    weight_diff = 0.0
    global_parameters = [param.detach().cpu().numpy() for param in global_model.parameters()]
    for global_param, client_param in zip(global_parameters, client_update):
        weight_diff += np.linalg.norm(global_param - client_param)
    return weight_diff


<font color='Brown'>***Clients Functions***</font>
---
--- 

In [100]:
def General_Client():
    def client_fn(cid: int, Round: int) -> FlowerClient:
        clients_ids_list = TrainingListPerRound[Round]
        if int(cid) in clients_ids_list:
            net = Net().to(DEVICE)
            trainloader = Dataloaders[Round][int(cid)]
            arg_ = Round
            return FlowerClient(cid, net, trainloader, arg_)
        else:
            raise ValueError(f"Client ID {cid} not found in the list for round {Round}")
    return client_fn

<font color='Brown'>***FL Strategy***</font>
---
--- 

In [101]:
Global_Models = {}
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def __init__(self, additional_argument, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.additional_argument = additional_argument
    def aggregate_fit(self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]], failures: List[BaseException]) -> Optional[fl.common.NDArrays]:
        aggregated_parameters_tuple = super().aggregate_fit(rnd, results, failures)
        aggregated_parameters, _ = aggregated_parameters_tuple
        if aggregated_parameters is not None:
            # print(f"Saving round {rnd} aggregated_parameters...")
            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_weights: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)
            # Convert `List[np.ndarray]` to PyTorch`state_dict`
            Global_Models[self.additional_argument] = Net()
            params_dict = zip(Global_Models[self.additional_argument].state_dict().keys(), aggregated_weights)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            Global_Models[self.additional_argument].load_state_dict(state_dict, strict=True)
            torch.save(Global_Models[self.additional_argument].state_dict(), f"{PATH}/GlobalModel_{self.additional_argument}.pth")
        return aggregated_parameters_tuple

In [102]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.
    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "current_round": server_round,  # The current round of federated learning
        "local_epochs": EPOCHS #1 if rnd < 2 else 2,  # 
    }
    return config

***Running Centralized DNN Rounds***
---
---

In [103]:

def train_one_round(model, trainloader, epochs):
    """Runs EPOCHS of training on this round's dataset."""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    model.train()

    for epoch in range(epochs):
        correct, total, running_loss = 0, 0, 0.0
        for Xbatch, ybatch in trainloader:
            Xbatch, ybatch = Xbatch.to(DEVICE), ybatch.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(Xbatch)
            loss = criterion(outputs, ybatch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            total += ybatch.size(0)
            correct += (torch.max(outputs, 1)[1] == ybatch).sum().item()

        print(f"  Epoch {epoch+1}/{epochs} | "
              f"Loss: {running_loss/len(trainloader):.6f} | "
              f"Acc: {correct/total:.4f}")


def test_model(model, testloader):
    """Runs a simple test pass (no saving)."""
    criterion = nn.CrossEntropyLoss()
    model.eval()
    correct, total, running_loss = 0, 0, 0.0

    with torch.no_grad():
        for Xbatch, ybatch in testloader:
            Xbatch, ybatch = Xbatch.to(DEVICE), ybatch.to(DEVICE)
            outputs = model(Xbatch)
            loss = criterion(outputs, ybatch).item()

            running_loss += loss
            total += ybatch.size(0)
            correct += (torch.max(outputs, 1)[1] == ybatch).sum().item()

    print(f"  Test → Loss: {running_loss/len(testloader):.6f}, "
          f"Acc: {correct/total:.4f}")


# ============================================================
# Centralized Training Loop
# ============================================================
if not FL:
    os.makedirs(PATH, exist_ok=True)
    print(" Starting centralized Round-by-Round training...\n")

    model = Net().to(DEVICE)

    for Round in range(1, ROUNDS + 1):

        print("========================")
        print(f"   ROUND {Round} START")
        print("========================")

        # Build round dataset from client partitions
        client_datasets = [
            loader.dataset for loader in Dataloaders[Round]
            if hasattr(loader, "dataset") and len(loader.dataset) > 0
        ]

        round_dataset = ConcatDataset(client_datasets)
        trainloader = DataLoader(round_dataset, batch_size=BATCH_SIZE, shuffle=True)

        print(f"  Train samples this round: {len(round_dataset)}")

        # Train
        train_one_round(model, trainloader, EPOCHS)

        # Evaluate (optional)
        print(f"  Evaluating round {Round} model...")
        test_model(model, Dataloaders["Test"])

        # Save model
        save_path = f"{PATH}/GlobalModel_{Round}.pth"
        torch.save(model.state_dict(), save_path)
        print(f"  💾 Saved: {save_path}")

        print("========================")
        print(f"   ROUND {Round} DONE")
        print("========================\n")

    print("🏁 Centralized Round-by-Round training completed.")


***Running the Generalized FL DNN Round***
---
---

In [104]:
if FL:
    os.makedirs(PATH, exist_ok=True)
    print("Loading Initial Global Model")
    Global_Models[0] = Net()
    Global_Models[0].load_state_dict(torch.load("0_Input_Random_model_Net.pth"))
    Global_Models[0].train()

    TrainingListPerRound = {}
    for Round in range(1, ROUNDS+1):
        TrainingListPerRound[Round] = []     
        for CLIENT in range (NUM_CLIENTS):
            TrainingListPerRound[Round].append(int(CLIENT))

    for Round in range(1, ROUNDS+1):
        print("Starting FL Round: ", Round)
        strategy = SaveModelStrategy(
                fraction_fit=1.0,  # Sample 100% of available clients for training
                fraction_evaluate=0,  # Sample 50% of available clients for evaluation
                min_fit_clients=2,  # Never sample less than 10 clients for training
                min_evaluate_clients=0,  # Never sample less than 5 clients for evaluation
                min_available_clients=2,  # Wait until all 10 clients are available
                on_fit_config_fn=fit_config,
                initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Global_Models[Round-1])),
                additional_argument = Round
        )
        # print(f'Current training nodes at round {Round}: ', len(TrainingListPerRound[Round]), ' ', TrainingListPerRound[Round])
        client_fn = General_Client()
        fl.simulation.start_simulation(
            client_fn=lambda cid: client_fn(cid, int(Round)),
            num_clients=int(NUM_CLIENTS),
            config=fl.server.ServerConfig(num_rounds=int(1)),
            client_resources={"num_cpus":1, "num_gpus":1}, 
            strategy=strategy
        )
        print("End of FL Round: ", Round)
        print("Loading Global Model: ", Round)
        Global_Models[Round] = Net()
        Global_Models[Round].load_state_dict(torch.load(f"{PATH}/GlobalModel_{Round}.pth"))
        Global_Models[Round].train()

INFO flwr 2026-01-07 01:52:24,885 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)


Loading Initial Global Model
Starting FL Round:  1


2026-01-07 01:52:29,824	INFO worker.py:2012 -- Started a local Ray instance.
INFO flwr 2026-01-07 01:52:36,272 | app.py:210 | Flower VCE: Ray initialized with resources: {'accelerator_type:T4': 1.0, 'object_store_memory': 2522483097.0, 'memory': 5885793895.0, 'CPU': 4.0, 'node:__internal_head__': 1.0, 'node:10.192.12.147': 1.0, 'GPU': 1.0}
INFO flwr 2026-01-07 01:52:36,274 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO flwr 2026-01-07 01:52:36,314 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO flwr 2026-01-07 01:52:36,318 | server.py:89 | Initializing global parameters
INFO flwr 2026-01-07 01:52:36,321 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2026-01-07 01:52:36,323 | server.py:91 | Evaluating initial parameters
INFO flwr 2026-01-07 01:52:36,324 | server.py:104 | FL starting
DEBUG flwr 2026-01-07 01:52:36,326 | server.py:222 | fit_round 1: strategy sampled 48 clients

(DefaultActor pid=73159) [Client 14, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=73159) Epoch 1/3 | Loss: 2.565588 | Acc: 0.5443
(DefaultActor pid=73159) Epoch 2/3 | Loss: 2.342136 | Acc: 0.6120
(DefaultActor pid=73159) Epoch 3/3 | Loss: 1.938359 | Acc: 0.6120
(DefaultActor pid=73159) [Client 39, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=73159) Epoch 1/3 | Loss: 2.519097 | Acc: 0.6927
(DefaultActor pid=73159) Epoch 2/3 | Loss: 2.260020 | Acc: 0.7292
(DefaultActor pid=73159) Epoch 3/3 | Loss: 1.788520 | Acc: 0.7292
(DefaultActor pid=73159) [Client 18, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=73159) Epoch 1/3 | Loss: 2.560408 | Acc: 0.5469
(DefaultActor pid=73159) Epoch 2/3 | Loss: 2.326181 | Acc: 0.6224
(DefaultActor pid=73159) Epoch 3/3 | Loss: 1.905205 | Acc: 0.6224
(DefaultActor pid=73159) [Client 30, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultAc

(pid=gcs_server) [2026-01-07 01:52:56,786 E 73030 73030] (gcs_server) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


...

(DefaultActor pid=125508) [Client 6, round 40] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=125508) Epoch 1/3 | Loss: 0.419165 | Acc: 0.8073
(DefaultActor pid=125508) Epoch 2/3 | Loss: 0.393588 | Acc: 0.8307
(DefaultActor pid=125508) Epoch 3/3 | Loss: 0.385858 | Acc: 0.8307
(DefaultActor pid=125508) [Client 31, round 40] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=125508) Epoch 1/3 | Loss: 0.243806 | Acc: 0.8750
(DefaultActor pid=125508) Epoch 2/3 | Loss: 0.215960 | Acc: 0.8958
(DefaultActor pid=125508) Epoch 3/3 | Loss: 0.213504 | Acc: 0.8984
(DefaultActor pid=125508) [Client 43, round 40] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=125508) Epoch 1/3 | Loss: 0.330110 | Acc: 0.8333
(DefaultActor pid=125508) Epoch 2/3 | Loss: 0.308443 | Acc: 0.8516
(DefaultActor pid=125508) Epoch 3/3 | Loss: 0.304480 | Acc: 0.8542
(DefaultActor pid=125508) [Client 29, round 40] fit, config: {'current_round': 1, 'local_epochs

(DefaultActor pid=125508) [2026-01-07 02:36:05,455 E 125508 125927] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
[2026-01-07 02:36:05,839 E 27558 125507] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


(DefaultActor pid=125508) [Client 20, round 40] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=125508) Epoch 1/3 | Loss: 0.325252 | Acc: 0.8385
(DefaultActor pid=125508) Epoch 2/3 | Loss: 0.292763 | Acc: 0.8490
(DefaultActor pid=125508) Epoch 3/3 | Loss: 0.286050 | Acc: 0.8776
(DefaultActor pid=125508) [Client 28, round 40] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=125508) Epoch 1/3 | Loss: 0.399271 | Acc: 0.8021
(DefaultActor pid=125508) Epoch 2/3 | Loss: 0.372789 | Acc: 0.8229
(DefaultActor pid=125508) Epoch 3/3 | Loss: 0.366817 | Acc: 0.8255
(DefaultActor pid=125508) [Client 30, round 40] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=125508) Epoch 1/3 | Loss: 0.058477 | Acc: 0.9714
(DefaultActor pid=125508) Epoch 2/3 | Loss: 0.042985 | Acc: 0.9792
(DefaultActor pid=125508) Epoch 3/3 | Loss: 0.037945 | Acc: 0.9818
(DefaultActor pid=125508) [Client 21, round 40] fit, config: {'current_round': 1, 'local_epoch

ERROR flwr 2026-01-07 02:36:33,020 | ray_client_proxy.py:147 | Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 140, in _submit_job
    res = self.actor_pool.get_client_result(self.cid, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 402, in get_client_result
    return self._fetch_future_result(cid)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 288, in _fetch_future_result
    res_cid, res = ray.get(future)  # type: (str, ClientRes)
                   ^^^^^^^^^^^^^^^
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
 

End of FL Round:  40
Loading Global Model:  40


<font color='Grey'>***Performance Testing***</font>
---
--- 

In [105]:
import os
import glob

# Define the directory and file pattern
directory = PATH + '/'
pattern = "GlobalModel_*.pth"

# Find all matching files
files = glob.glob(os.path.join(directory, pattern))

# Extract numbers from file names
numbers = []
for file in files:
    base_name = os.path.basename(file)
    num_str = base_name.replace("GlobalModel_", "").replace(".pth", "")
    try:
        numbers.append(int(num_str))
    except ValueError:
        pass

# Determine the maximum number
max_num = max(numbers) if numbers else 0
print(max_num)

# Use the max_num in a loop
for num in range(1, max_num + 1):
    file_path = f"{PATH}/GlobalModel_{num}.pth"
    if os.path.exists(file_path):
        # Load the file or perform any operation you need
        print(f"Loading {file_path}")
    else:
        print(f"File {file_path} does not exist")

40
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_1.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_2.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_3.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_4.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_5.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_6.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_7.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_8.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_9.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-500-groups/GlobalModel_10.pth
Loading FL:TrueDNN48-clients-11atk-40-rounds-3-epochs-0.0025-lr-50

In [106]:
def view_pickle(path, G=None, suffix=None):
    print(G)
    if G is not None:
        rounds = [G]
    else:
        # Auto-detect how many Global_X files exist
        rounds = sorted({
            int(f.split("_")[1])
            for f in os.listdir(path)
            if f.startswith("Global_") and f.split("_")[1].isdigit()
        })

    suffixes = [suffix] if suffix else ["pred", "actual", "accurracy", "loss"]

    for g in rounds:
        for s in suffixes:
            filename = os.path.join(path, f"Global_{g}_{s}")
            if not os.path.exists(filename):
                print(f"Missing file: {filename}")
                continue
            try:
                with open(filename, "rb") as f:
                    data = pickle.load(f)
                print(f"File: {filename}")
                if isinstance(data, list) and len(data) > 2:
                    pprint(data[:2])
                    print(f"... ({len(data)} total items)")
                else:
                    pprint(data)
            except Exception as e:
                print(f"❌ Error reading {filename}: {e}")



In [107]:
pred_test = {}
actual_test = {}
accuracy_test = {}
loss_test = {}
G = 0

for num in range(1, max_num+1):
    model = Net()
    model.load_state_dict(torch.load(f"{PATH}/GlobalModel_{num}.pth"))
    model.eval()
    
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    G = G + 1
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in Dataloaders['Test']:
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    loss /= len(Dataloaders['Test'].dataset)
    accuracy = correct / total
    loss_matrix.append(loss)
    acc_matrix.append(accuracy) 

    pred_test[f'Global_{G}'] = prediction_matrix
    actual_test[f'Global_{G}'] = actual_matrix
    accuracy_test[f'Global_{G}'] = acc_matrix
    loss_test[f'Global_{G}'] = loss_matrix 

    filename = f'{PATH}/Global_{G}_pred'
    outfile = open(filename,'wb')
    pickle.dump(pred_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_actual'
    outfile = open(filename,'wb')
    pickle.dump(actual_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_accurracy'
    outfile = open(filename,'wb')
    pickle.dump(accuracy_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_loss'
    outfile = open(filename,'wb')
    pickle.dump(loss_test[f'Global_{G}'],outfile)
    outfile.close()

(raylet) [2026-01-07 02:37:28,562 E 125464 125464] (raylet) node_manager.cc:3252: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 630dda0486dc2ef274f93e3be50d1c0244cb537f1ff1036d84af9fd4, IP: 10.192.12.147) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.192.12.147`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
